In [ ]:
import twint
import pandas as pd

# sentiment analysis
from optimus import Optimus
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

op = Optimus()

# For notebooks 
import nest_asyncio
nest_asyncio.apply()

https://towardsdatascience.com/analyzing-tweets-with-nlp-in-minutes-with-spark-optimus-and-twint-a0c96084995f

In [ ]:
#configuration
FILENAME = 'tweets.json'

config = twint.Config()
config.Search = "bitcoin"
#config.Lang = "en"
config.Limit = 1
# config.Since = "2019-04-29 00:00:00" #datetime.strptime("2019-04-29 00:00:00", '%Y-%m-%d %H:%M:%S')
#config.Until = "2020-04-29 12:00:00" # datetime.strptime("2020–04–29 12:00:00", '%Y-%m-%d %H:%M:%S')
#config.Min_likes = 500
config.Pandas = True
config.Hide_output = True

#running search
twint.run.Search(config)

In [ ]:
#https://github.com/FavioVazquez/twitter_optimus_twint/blob/master/AnalyzeTweets.ipynb

# return all the columns in the twint output
def available_columns():
    return twint.output.panda.Tweets_df.columns

# return only the needed columns for analysis
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

# Function to get sentiment
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0  # Neutral
    elif temp >= 0.0:
        return 1.0  # Positive
    else:
        return 2.0  # Negative

# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())

def create_optimus(df):
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf=df)

    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet").cols.remove_special_chars("tweet")

    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment", sentiment(clean_tweets['tweet']))

In [ ]:
print(available_columns())

df = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
df.head()

In [ ]:
# create the Optimus DataFrame 
op_df = create_optimus(df)

In [ ]:
# ERROR: Java error,
# IllegalArgumentException: 'Unsupported class file major version 57'
op_df.show()

In [ ]:
op_df.printSchema()

## THIS SECTION IS FOR WHEN THE TWITTER DATA IS SAVED AS A JSON FILE
# config.Store_json = True 

In [ ]:
# modifying json file
import json

# Reading the json
with open(FILENAME, encoding="utf8") as json_data:
    data = json.load(json_data)
    
    # store the data as a pandas dataframe
    df= pd.DataFrame(data["tweets"])

# Drop all the columns and rows that conatin null values
df.dropna(axis=0, how="any", inplace=False)

# drop all the rows that do not affect the sentiment
unwanted_cols = ["id", "created_at", "conversation_id", "timezone", "urls", "photos", "thumbnail", "near", "geo", "user_rt_id", "user_rt", "retweet_id",
                 "reply_to", "translate", "trans_src", "trans_dest", "cashtags", "retweet_date", "retweet", "video", "source", "quote_url", "place"]
df.drop(unwanted_cols, axis=1, inplace=True)

# create a new Date column consisting of a concatinated string of the date and time columns
# drop the date and time columns from the dataframe
# and set the Date column as the index of the dataframe
df['Date'] = df['date'].str.cat(df['time'], sep=" ")
df.drop(["date", "time"], axis=1, inplace=True)
df.set_index('Date', inplace=True)

#print(df.columns)
df.head()